In [1]:
!pip3 install transformers

  Using cached https://files.pythonhosted.org/packages/13/33/ffb67897a6985a7b7d8e5e7878c3628678f553634bd3836404fef06ef19b/transformers-2.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/3f/73c881ea4723e43c1e9acf317cf407fab3a278daab3a69c98dcac511c04f/tokenizers-0.5.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/74/f4/2d5214cbf13d06e7cb2c20d84115ca25b53ea76fa1f0ade0e3c9749de214/sentencepiece-0.1.85-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ed/36/fd20c656fb4a4fbe8db367ea274c3465b81cb2e01ffc57b9980f0578e131/regex-2020.2.20-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/47/55/fd9170ba08a1a64a18a7f8a18f088037316f2a41be04d2fe6ece5a653e8f/tqdm-4.43.0-py2.py3-none-any.whl


In [2]:
import torch

In [3]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
def ask_BERT(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [6]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

In [7]:
George_Wasington_bio = "George Washington (February 22, 1732[b] – December 14, 1799) was an American political leader, military general, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Previously, he led Patriot forces to victory in the nation's War for Independence. He presided at the Constitutional Convention of 1787, which established the U.S. Constitution and a federal government. Washington has been called the 'Father of His Country' for his manifold leadership in the formative days of the new nation.\
Washington received his initial military training and command with the Virginia Regiment during the French and Indian War. He was later elected to the Virginia House of Burgesses and was named a delegate to the Continental Congress, where he was appointed Commanding General of the Continental Army. He commanded American forces, allied with France, in the defeat and surrender of the British during the Siege of Yorktown. He resigned his commission after the Treaty of Paris in 1783.\
Washington played a key role in adopting and ratifying the Constitution and was then elected president (twice) by the Electoral College. He implemented a strong, well-financed national government while remaining impartial in a fierce rivalry between cabinet members Thomas Jefferson and Alexander Hamilton. During the French Revolution, he proclaimed a policy of neutrality while sanctioning the Jay Treaty. He set enduring precedents for the office of president, including the title 'President of the United States', and his Farewell Address is widely regarded as a pre-eminent statement on republicanism.\
Washington owned slaves, and in order to preserve national unity he supported measures passed by Congress to protect slavery. He later became troubled with the institution of slavery and freed his slaves in a 1799 will. He endeavored to assimilate Native Americans into Anglo-American culture but combated indigenous resistance during occasions of violent conflict. He was a member of the Anglican Church and the Freemasons, and he urged broad religious freedom in his roles as general and president. Upon his death, he was eulogized as 'first in war, first in peace, and first in the hearts of his countrymen'. He has been memorialized by monuments, art, geographical locations, stamps, and currency, and many scholars and polls rank him among the greatest U.S. presidents."

print(wrapper.fill(George_Wasington_bio))

George Washington (February 22, 1732[b] – December 14, 1799) was an American
political leader, military general, statesman, and Founding Father who served as
the first president of the United States from 1789 to 1797. Previously, he led
Patriot forces to victory in the nation's War for Independence. He presided at
the Constitutional Convention of 1787, which established the U.S. Constitution
and a federal government. Washington has been called the 'Father of His Country'
for his manifold leadership in the formative days of the new nation.Washington
received his initial military training and command with the Virginia Regiment
during the French and Indian War. He was later elected to the Virginia House of
Burgesses and was named a delegate to the Continental Congress, where he was
appointed Commanding General of the Continental Army. He commanded American
forces, allied with France, in the defeat and surrender of the British during
the Siege of Yorktown. He resigned his commission after 

In [9]:
question = "Who was the first president?"

ask_BERT(question, George_Wasington_bio)

Answer: "george washington"


In [11]:
question = "When was George Washinton Born?"

ask_BERT(question, George_Wasington_bio)

Answer: "february 22 , 1732"


In [12]:
question = "When did George Washinton Die?"

ask_BERT(question, George_Wasington_bio)

Answer: "december 14 , 1799"


In [13]:
question = "What war was George Washington in?"

ask_BERT(question, George_Wasington_bio)

Answer: "war for independence"


In [14]:
question = "What Cabinet members had a fierce rivalry?"

ask_BERT(question, George_Wasington_bio)

Answer: "thomas jefferson and alexander hamilton"


In [15]:
question = "How was George Washington eulogized?"

ask_BERT(question, George_Wasington_bio)

Answer: "' first in war , first in peace , and first in the hearts of his countrymen"


In [16]:
SB_LIV_story ="Super Bowl LIV was an American football game played to determine the champion of the National Football League (NFL) for the 2019 season. The American Football Conference (AFC) champion Kansas City Chiefs defeated the National Football Conference (NFC) champion San Francisco 49ers 31–20, marking their first Super Bowl victory since Super Bowl IV and the Chiefs' first NFL championship since joining the league in the AFL–NFL merger. Chiefs quarterback Patrick Mahomes won the Super Bowl Most Valuable Player Award. Mahomes completed 26 of 42 pass attempts for 286 yards (two touchdowns, two interceptions) and rushed for 29 yards (one touchdown).[8]\
The game was played on February 2, 2020, at Hard Rock Stadium in Miami Gardens, Florida. This was the 11th Super Bowl hosted by the South Florida region and the sixth Super Bowl hosted in Miami Gardens, which hosted Super Bowl XLIV ten years earlier.\
The first half of the game was back and forth between both teams, with the game tied at 10 heading into halftime. In the third quarter, the 49ers began to pull away, with a Robbie Gould field goal and a rushing touchdown by Raheem Mostert giving them a 20–10 lead heading into the fourth quarter. However, over the last 6:13 of the game, the Chiefs offense completed two touchdown drives with Mahomes throwing touchdown passes to Travis Kelce and Damien Williams, to take the lead just before the two-minute warning. The Chiefs then stopped the 49ers on defense, and a late rushing touchdown by Damien Williams, as well as a Kendall Fuller interception, sealed the victory and ended the Chiefs' 50-year championship drought.[9]\
The game's broadcast in the United States by Fox, along with the halftime show headlined by Jennifer Lopez and Shakira, was seen by an estimated 102 million viewers—a slight increase over Super Bowl LIII in 2019 (which had seen the smallest audience for the game in 10 years)."

print(wrapper.fill(SB_LIV_story))

Super Bowl LIV was an American football game played to determine the champion of
the National Football League (NFL) for the 2019 season. The American Football
Conference (AFC) champion Kansas City Chiefs defeated the National Football
Conference (NFC) champion San Francisco 49ers 31–20, marking their first Super
Bowl victory since Super Bowl IV and the Chiefs' first NFL championship since
joining the league in the AFL–NFL merger. Chiefs quarterback Patrick Mahomes won
the Super Bowl Most Valuable Player Award. Mahomes completed 26 of 42 pass
attempts for 286 yards (two touchdowns, two interceptions) and rushed for 29
yards (one touchdown).[8]The game was played on February 2, 2020, at Hard Rock
Stadium in Miami Gardens, Florida. This was the 11th Super Bowl hosted by the
South Florida region and the sixth Super Bowl hosted in Miami Gardens, which
hosted Super Bowl XLIV ten years earlier.The first half of the game was back and
forth between both teams, with the game tied at 10 heading i

In [17]:
question = "Who won SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "kansas city chiefs"


In [18]:
question = "Who lost SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "san francisco 49ers"


In [19]:
question = "What was the score of SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "31 – 20"


In [20]:
question = "How many Touchdowns were scored in SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "two"


In [21]:
question = "Who was the MVP in SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "patrick mahomes"


In [22]:
question = "What did Robbie Gould do in SuperBowl LIV?"

ask_BERT(question, SB_LIV_story)

Answer: "field goal"


In [24]:
with open("movies-sequence-input.txt", "r", encoding="utf-8") as file_input:
    movie_input = file_input.read()


In [26]:
question = "Hello how are you"

ask_BERT(question, movie_input[0:470])

Answer: "cameron"


In [27]:
question = "Will you marry me?"

ask_BERT(question, movie_input[0:470])

Answer: "cameron"


In [28]:
question = "The sun is hot"

ask_BERT(question, movie_input[0:470])

Answer: "cameron"
